<a href="https://colab.research.google.com/github/Rodrigo-Perico/Introduction-to-Machine-Learning-on-AWS/blob/main/SVM_teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install optuna
!pip install ucimlrepo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.0 MB/s eta 0:00:00


In [121]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import optuna
from plotly.io import show
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, FunctionTransformer, RobustScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, f1_score, make_scorer, confusion_matrix, ConfusionMatrixDisplay, recall_score, precision_score
from sklearn.compose import ColumnTransformer
from ucimlrepo import fetch_ucirepo
from functools import partial
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from imblearn.over_sampling import *
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Importando a Base de dados e separando 5%



In [122]:
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

data = cdc_diabetes_health_indicators.data.features
aux = cdc_diabetes_health_indicators.data.targets

data["Diabetes_binary"] = aux["Diabetes_binary"]

#print(cdc_diabetes_health_indicators.metadata)

#print(cdc_diabetes_health_indicators.variables)

data.columns

<ipython-input-122-0eb90a80a6c7>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income',
       'Diabetes_binary'],
      dtype='object')

In [123]:
data['Age']=data['Age'].astype('int64')
data['Education']=data['Education'].astype('int64')
data['Income']=data['Income'].astype('int64')
data['BMI']=data['BMI'].astype('int64')
data['CholCheck']=data['CholCheck'].astype('int64')
data['Smoker']=data['Smoker'].astype('int64')
data['Stroke']=data['Stroke'].astype('int64')
data['HeartDiseaseorAttack']=data['HeartDiseaseorAttack'].astype('int64')
data['PhysActivity']=data['PhysActivity'].astype('int64')
data['Fruits']=data['Fruits'].astype('int64')
data['Veggies']=data['Veggies'].astype('int64')
data['HvyAlcoholConsump']=data['HvyAlcoholConsump'].astype('int64')
data['AnyHealthcare']=data['AnyHealthcare'].astype('int64')
data['NoDocbcCost']=data['NoDocbcCost'].astype('int64')
data['GenHlth']=data['GenHlth'].astype('int64')
data['MentHlth']=data['MentHlth'].astype('int64')
data['PhysHlth']=data['PhysHlth'].astype('int64')
data['DiffWalk']=data['DiffWalk'].astype('int64')
data['Sex']=data['Sex'].astype('int64')
data['Diabetes_binary']=data['Diabetes_binary'].astype('int64')
data['HighBP']=data['HighBP'].astype('int64')
data['HighChol']=data['HighChol'].astype('int64')

<ipython-input-123-bc92d329addf>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-123-bc92d329addf>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-123-bc92d329addf>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-123-bc9

In [124]:
data.duplicated().sum()

data.drop_duplicates(inplace=True)

<ipython-input-124-1ecf93f2f3f5>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [125]:
categorical_columns = ['GenHlth', 'Age', 'Education', 'Income']
numerical_columns = ['BMI', 'MentHlth', 'PhysHlth']
binary_columns = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']

In [126]:
data1 = data.sample(frac=0.1, random_state=42)

x = data1.drop('Diabetes_binary', axis = 1)
y = data1['Diabetes_binary']

In [127]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.15, random_state=42, stratify=y_train)


In [81]:
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.transform(x_test)

In [117]:
preprocessor_svm = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(sparse_output= False, feature_name_combiner = 'concat'), categorical_columns),
        ('bin', 'passthrough', binary_columns),
    ])


pipeline = Pipeline(steps=[
      ('preprocessor_svm', preprocessor_svm),
      ('model', SVC())
      #('model', model)
      #('outlier_remover', outlier_remover),
      #('svm' , SVC(kernel='sigmoid', gamma = 0.01, C=733)),
  ])

In [132]:
teste1 = preprocessor_svm.fit_transform(x_train)

print(teste1.shape)
print(teste1)

# teste1=pd.DataFrame(teste1)
# teste1.head(10)
features = preprocessor_svm.get_feature_names_out()
print(features)
# print(categorical_columns)


(15603, 46)
[[0. 0. 1. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 ...
 [1. 0. 0. ... 0. 1. 0.]
 [0. 0. 1. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 1. 1.]]
['cat__GenHlth_1' 'cat__GenHlth_2' 'cat__GenHlth_3' 'cat__GenHlth_4'
 'cat__GenHlth_5' 'cat__Age_1' 'cat__Age_2' 'cat__Age_3' 'cat__Age_4'
 'cat__Age_5' 'cat__Age_6' 'cat__Age_7' 'cat__Age_8' 'cat__Age_9'
 'cat__Age_10' 'cat__Age_11' 'cat__Age_12' 'cat__Age_13'
 'cat__Education_1' 'cat__Education_2' 'cat__Education_3'
 'cat__Education_4' 'cat__Education_5' 'cat__Education_6' 'cat__Income_1'
 'cat__Income_2' 'cat__Income_3' 'cat__Income_4' 'cat__Income_5'
 'cat__Income_6' 'cat__Income_7' 'cat__Income_8' 'bin__HighBP'
 'bin__HighChol' 'bin__CholCheck' 'bin__Smoker' 'bin__Stroke'
 'bin__HeartDiseaseorAttack' 'bin__PhysActivity' 'bin__Fruits'
 'bin__Veggies' 'bin__HvyAlcoholConsump' 'bin__AnyHealthcare'
 'bin__NoDocbcCost' 'bin__DiffWalk' 'bin__Sex']


###                                Oversampling

In [127]:
over_methods = [
    ADASYN(random_state=7),
    BorderlineSMOTE(random_state=7),
    RandomOverSampler(random_state=7),
    SMOTE(random_state=7),
    SVMSMOTE(random_state=7)
]

In [128]:
classifiers =[
    SVC()
]

Metricas para comparar os métodos

In [129]:
metrics = {
    'roc_auc': 'roc_auc',
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}


In [130]:
steps = [('model', SVC(kernel='sigmoid', C=733, gamma= 0.01  ))]
pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=7)

for metric_name, metric in metrics.items():
    scores = cross_val_score(
        pipeline, x_train, y_train, scoring=metric, cv=cv, n_jobs=-1
    )
    print(f"{metric_name.upper()} for SVM is {round(np.mean(scores), 5)}")


KeyboardInterrupt: 

In [ ]:
for classifier in classifiers:
    for method in over_methods:
        X_resampled, y_resampled = method.fit_resample(X, y)

        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('classifier', classifier)
        ])

        cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=7)

        for metric_name, metric in metrics.items():
            scores = cross_val_score(
                pipeline, X_resampled, y_resampled, scoring=metric, cv=cv, n_jobs=-1
            )
            print(f"{metric_name.upper()} for {classifier.__class__.__name__} and {method.__class__.__name__} is {round(np.mean(scores), 5)}")


ROC_AUC for SVC and ADASYN is 0.86367
ACCURACY for SVC and ADASYN is 0.78555
PRECISION for SVC and ADASYN is 0.76707
RECALL for SVC and ADASYN is 0.81827
F1 for SVC and ADASYN is 0.79182
ROC_AUC for SVC and BorderlineSMOTE is 0.89333
ACCURACY for SVC and BorderlineSMOTE is 0.81564
PRECISION for SVC and BorderlineSMOTE is 0.78962
RECALL for SVC and BorderlineSMOTE is 0.86064
F1 for SVC and BorderlineSMOTE is 0.82357
ROC_AUC for SVC and RandomOverSampler is 0.8769
ACCURACY for SVC and RandomOverSampler is 0.8007
PRECISION for SVC and RandomOverSampler is 0.77309
RECALL for SVC and RandomOverSampler is 0.85131
F1 for SVC and RandomOverSampler is 0.81029
ROC_AUC for SVC and SMOTE is 0.87505
ACCURACY for SVC and SMOTE is 0.79776
PRECISION for SVC and SMOTE is 0.78069
RECALL for SVC and SMOTE is 0.82831
F1 for SVC and SMOTE is 0.80376
ROC_AUC for SVC and SVMSMOTE is 0.90517
ACCURACY for SVC and SVMSMOTE is 0.8292
PRECISION for SVC and SVMSMOTE is 0.81418
RECALL for SVC and SVMSMOTE is 0.8531

## TESTANDO ACURÁCIA ANTES DE APLICAR O BORDERLINESMOTE

In [131]:
model = SVC()
pipeline = make_pipeline( model)

pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)

print(f'Acurácia sem aplicar o SMOTE: {accuracy}')

Acurácia sem aplicar o SMOTE: 0.8488942150561063


In [132]:
x_train.shape

(31208, 21)

In [133]:
method = BorderlineSMOTE(random_state=7)
x_train_resampled, y_train_resampled = method.fit_resample(x_train, y_train)

x_train, y_train = x_train_resampled, y_train_resampled

## TESTANDO ACURÁCIA DEPOIS DE APLICAR O BORDERLINESMOTE

In [134]:
model = SVC()
pipeline = make_pipeline( model)

pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)

print(f'Acurácia aplicando o SMOTE: {accuracy}')

Acurácia aplicando o SMOTE: 0.7414751062207212


In [135]:
x_train.shape

(52784, 21)

In [136]:
y_train.value_counts()

,count
Diabetes_binary,
1,26392
0,26392


## Testando acurácia antes de aplicar o PCA

In [137]:
column_names = numerical_columns + categorical_columns + binary_columns

x_train = pd.DataFrame(x_train, columns=column_names)
x_test = pd.DataFrame(x_test, columns=column_names)
x_validation = pd.DataFrame(x_validation, columns=column_names)

In [116]:
pipeline = make_pipeline(model)

pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia com todas as FEATURES: {accuracy}')

Acurácia com todas as FEATURES: 0.7577342047930283


In [117]:
for i in range(1, 22):
    print("==========================================================")

    pca = PCA(n_components=i)

    pipeline = make_pipeline(pca, model)

    pipeline.fit(x_train, y_train)

    y_pred = pipeline.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)

    print(f'Acurácia com PCA ({i} componentes): {accuracy}')

Acurácia com PCA (1 componentes): 0.677124183006536
Acurácia com PCA (2 componentes): 0.69760348583878
Acurácia com PCA (3 componentes): 0.6958605664488018
Acurácia com PCA (4 componentes): 0.7132897603485838
Acurácia com PCA (5 componentes): 0.7185185185185186
Acurácia com PCA (6 componentes): 0.7250544662309368
Acurácia com PCA (7 componentes): 0.7376906318082789
Acurácia com PCA (8 componentes): 0.739433551198257
Acurácia com PCA (9 componentes): 0.7355119825708061
Acurácia com PCA (10 componentes): 0.739433551198257
Acurácia com PCA (11 componentes): 0.7398692810457517
Acurácia com PCA (12 componentes): 0.742483660130719
Acurácia com PCA (13 componentes): 0.7437908496732026
Acurácia com PCA (14 componentes): 0.743355119825708
Acurácia com PCA (15 componentes): 0.7468409586056645
Acurácia com PCA (16 componentes): 0.7525054466230937
Acurácia com PCA (17 componentes): 0.7546840958605664
Acurácia com PCA (18 componentes): 0.7538126361655774
Acurácia com PCA (19 componentes): 0.7533769

In [98]:
def objective(trial):
    pipeline = Pipeline([
        ('preprocessor', preprocessor_svm),
        ('svm', SVC())
    ])

    C = trial.suggest_float('C', 1.0, 1000.0, log=True)  # C em escala logarítmica
    gamma = trial.suggest_float('gamma', 0.01, 2.0, step=0.005)  # Gamma
    kernel = trial.suggest_categorical('kernel', ['rbf', 'poly', 'sigmoid'])  # Kernel

    # Pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor_svm),
        ('svm', SVC())
    ])
    pipeline.set_params(svm__C=C, svm__gamma=gamma, svm__kernel=kernel)

    # Treinamento e predição
    pipeline.fit(x_train, y_train)
    y_pred = pipeline.predict(x_validation)

    # Métrica de avaliação
    score = f1_score(y_validation, y_pred)
    return score


# Criação do estudo e execução
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Alterar n_trials conforme necessário

# Exibição dos melhores parâmetros
print("Melhores parâmetros:", study.best_params)

[I 2024-11-24 14:30:33,277] A new study created in memory with name: no-name-1e7f1745-c41e-42c5-91cd-6ed9213ee918
[W 2024-11-24 14:30:33,287] Trial 0 failed with parameters: {'C': 32.06716340019006, 'gamma': 1.555, 'kernel': 'sigmoid'} because of the following error: ValueError('A given column is not a column of the dataframe').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_indexing.py", line 361, in _get_column_indices
    col_idx = all_columns.get_loc(col)
  File "/usr/local/lib/python3.10/dist-packages/pandas/core/indexes/range.py", line 417, in get_loc
    raise KeyError(key)
KeyError: 'GenHlth'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-98-6e2b516aafa7>", line 19, in objective
    pipeline.fit(x_train, y_train)
  

ValueError: A given column is not a column of the dataframe

In [ ]:
pipeline = Pipeline([
      #('outlier_remover', outlier_remover),
      ('preprocessor_svm', preprocessor_svm),
      ('svm', SVC())
  ])

C = 900
gamma = 0.11215038461268562
kernel=  'sigmoid'

pipeline.set_params(svm__C = C, svm__gamma = gamma, svm__kernel = kernel)
pipeline.fit(x_train, y_train)

y_pred = pipeline.predict(x_temp)
score = f1_score(y_temp, y_pred)
print(score)
print(classification_report(y_temp, y_pred))


SyntaxError: incomplete input (<ipython-input-67-81cfc10a30d3>, line 1)

In [ ]:
print(score)

0.1935483870967742


In [ ]:
print(confusion_matrix(y_temp, y_pred))



[[124   8]
 [ 17   3]]
